# Semigroup and Monoid

## Outline

* Definition of Semigroup

* Semigroup example

* Definition of Monoid

* Monoid examples

In this lesson, we will learn about the Semigroup and Monoid type classes and how you can use them.

## Definition of Semigroup
Semigroup is a type class that provides the operator <> that is used for combining variables of same type. Here is the type signature:
```haskell
(<>) :: Semigroup a => a -> a -> a
```

Some of the Haskell types have an instance of Semigroup as lists [] for example. The <> operator behaves same as the ++ operator.
```haskell
[1] <> [2] -- gives you [1,2]
"a" <> "b" -- gives you "ab"
```

Or you could implement Semigroup for an existing type as Integer:
```haskell
instance Semigroup Integer where
   (<>) x y = x + y
```

After this instance if you say `3 + 4` you will get the same result as if you say `3 <> 4`.

## Semigroup example

Lets define our own type called Game where we define 4 options: Rock, Paper, Scissors and Mistake. When mixing them we play by standard rules (https://en.wikipedia.org/wiki/Rock_paper_scissors) only we add the Mistake element that is the case where someone did not made a clear coice. And mixing Mistake with any of the other 3 choices gives the other choice. 

In [ ]:
data Game = Rock | Paper | Scissors | Mistake deriving (Show, Eq)

instance Semigroup Game where
    (<>) Rock Paper = Paper
    (<>) Paper Rock = Paper
    (<>) Rock Scissors = Rock
    (<>) Scissors Rock = Rock
    (<>) Paper Scissors = Scissors
    (<>) Scissors Paper = Scissors
    (<>) a b | a == b = a
             | a == Mistake = b
             | b == Mistake = a

c1, c2, c3 :: Game
c1 = Rock
c2 = Paper
c3 = Mistake

print $ c1 <> c2
print $ c2 <> c2
print $ c1 <> c3
print $ c3 <> c3

## Definition of Monoid

Monoid is an extension of the Semigroup type class, so it is also used for combining variables of same type. Monoids definition is:
```haskell
class Semigroup a => Monoid a where
  mempty :: a
  mappend :: a -> a -> a
  mconcat :: [a] -> a
```

The `mappend` function has the same type as the `<>` operator from Semigroup, so if you define an instace for Semigroup you can simply say `mappend = <>`. The `mempty` operator is used as a neutral element which means that combined with any other element it does not change the element. `mconcat` is used for combining multiple elements together with `mapend`. The minimal complete definition for the Monoid type class are only the first two operators.

Lists are also an instance of Monoid so you can use the Monoid operators as follows:
```haskell
[1] `mappend` [2] -- gives you [1,2]
"a" `mappend` "b" -- gives you "ab"
[1] `mappend` mempty -- gives you [1]
mconcat ["a","b","c"] -- gives you "abc"
```

Here is how you would implement Monoid for Integer if a Semigroup instance already exists:
```haskell
instance Monoid Integer where
    mempty = 0
    mappend = (<>)
```

There are found laws that govern the Monoid type class:

* mappend mempty x = x.

* mappend x mempty = x.

* mappend x (mappend y z) = mappend (mappend x y) z.

* mconcat = foldr mappend mempty

You have to take care of the first three when you implement a Monoid instance. The fourth is just a definition of mconcat that you get for free.

## Monoid examples

Lets pick up on the Semigroup example and try to implement a Monoid class for it.

In [ ]:
instance Monoid Game where
    mempty = Mistake
    mappend = (<>)

print $ mconcat [Paper, Mistake, Rock, Scissors, Mistake] 

Notice that the previous code returns Paper becase `mconcat [Paper, Mistake, Rock, Scissors, Mistake]` is equal to `Paper <> (Mistake <> (Rock <> (Scissors <> Mistake)))`. You can also directly define the mappend operations instead of defining the <> operations in the Semigroup class.

Lets look now at another example. You have 6 sided cube where you have the number 1 one time, the number 2 two times and number 3 three times on it. Now lets create a Cube type with these three options and an instance of Semigroup and Monoid that tells you the probability of the event when you combine two possible number outcomes. 

In [ ]:
data CubeProbability = CProb String Rational

instance Show CubeProbability where
    show (CProb a b) = show a ++ "|" ++ show (fromIntegral (round (fromRational b * 100000)) / 1000) ++ "%"

instance Semigroup CubeProbability where
    (<>) cprob1 (CProb "" 0) = cprob1
    (<>) (CProb "" 0) cprob2 = cprob2 
    (<>) (CProb num1 prob1) (CProb num2 prob2) = CProb (num1 <> "-" <> num2) (prob1*prob2)

instance Monoid CubeProbability where
    mempty = CProb "" 0
    mappend = (<>)

data Cube = One | Two | Three

cube2CProb :: Cube -> CubeProbability
cube2CProb cube = case cube of 
    One -> CProb "1" (1/6)
    Two -> CProb "2" (1/3)
    Three -> CProb "3" (1/2)

case1 :: [Cube]
case1 = [Two, One, One, Three, Two]

showProbability :: [Cube] -> CubeProbability
showProbability scenario = mconcat $ map cube2CProb scenario

-- Try defining a case two and see its probability
print $ showProbability case1

## Recap

In this lesson we've discussed:

- the definition and usage of Semigroup type class
- the definition and usage of Monoid type class